# 7 数据清洗与准备
重点讨论缺失值、重复值、字符串操作和其他数据转换工具

## 7.1 处理缺失值

In [1]:
# pandas对象的所有描述性统计信息默认情况下是排除缺失值的
import pandas as pd
import numpy as np

string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# NA意思是not available（不可用）
# 当数据清洗时，对缺失数据本身进行分析已确定数据收集问题或数据丢失导致的数据偏差通常很重要。
# None在对象数组中也被当作NA处理
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 过滤缺失值

In [6]:
from numpy import nan as NA

data = pd.Series([NA, 1, 2, NA, 4])
# Series中使用dropna，会返回所有非NA的数据
data.dropna()

1    1.0
2    2.0
4    4.0
dtype: float64

In [8]:
data[data.notnull()]

1    1.0
2    2.0
4    4.0
dtype: float64

In [11]:
# dropna默认删除包含NA的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.55, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
# 当传入how='all'时，会删除所有值均为NA的行
data.dropna(how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
3,NaN,6.55,3.0


In [15]:
data[4] = NA

In [16]:
data

,0,1,2,4
0,1.0,6.50,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.55,3.0,NaN


In [18]:
# 删除列加上axis=1
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [19]:
# 只想保留一定数量的行，使用thresh参数来表示
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,2.220267,NaN,NaN
1,1.648893,NaN,NaN
2,1.502815,NaN,-1.576268
3,-0.791778,NaN,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [20]:
df.dropna()

,0,1,2
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [21]:
df.dropna(thresh=2)

,0,1,2
2,1.502815,NaN,-1.576268
3,-0.791778,NaN,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


### 7.1.2 补全缺失值

In [23]:
# 大多数情况下使用fillna来不全缺失值
df.fillna(0)

,0,1,2
0,2.220267,0.000000,0.000000
1,1.648893,0.000000,0.000000
2,1.502815,0.000000,-1.576268
3,-0.791778,0.000000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [24]:
# 可以使用字典来为不同的列填充不同的值
df.fillna({1: 0.5, 2: 1})

,0,1,2
0,2.220267,0.500000,1.000000
1,1.648893,0.500000,1.000000
2,1.502815,0.500000,-1.576268
3,-0.791778,0.500000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [27]:
# fillna默认不会修改原有的对象，加inplace=True会就地修改
_ = df.fillna({1: 0.5, 2: 1}, inplace=True)
df

,0,1,2
0,2.220267,0.500000,1.000000
1,1.648893,0.500000,1.000000
2,1.502815,0.500000,-1.576268
3,-0.791778,0.500000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [28]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,NaN,-0.688223
3,-1.758211,NaN,1.070885
4,-0.309773,NaN,NaN
5,-0.772189,NaN,NaN


In [30]:
# 使用前一项值来填充空值
df.fillna(method='ffill')

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,-1.810273,-0.688223
3,-1.758211,-1.810273,1.070885
4,-0.309773,-1.810273,1.070885
5,-0.772189,-1.810273,1.070885


In [31]:
# 也可以使用均值等来填充缺失值
df.fillna(df.mean())

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,-0.762864,-0.688223
3,-1.758211,-0.762864,1.070885
4,-0.309773,-0.762864,-0.782728
5,-0.772189,-0.762864,-0.782728
